In [1]:
# import packages
from ast import literal_eval

# math and data packages
import pandas as pd
import numpy as np
import math

# charting and graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
# from matplotlib import colors
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg

# os and file types
import os
import sys
import datetime as dt
import json
import csv

In [2]:
# load invasive_preproc file and transform list data stored as str in csv into lists for new df

inv = pd.read_csv('output/invasive_preproc.csv')
newinv = inv[['Latein', 'Black List / Watch List 2014','Ordonnonce sur la dissémination des organismes (ODE)']].copy()
columns = ['threats','places','habitat']
for i in columns:
    newinv[i] = inv[[i]].applymap(literal_eval)
inv = newinv

In [3]:
df = pd.read_csv('output/alldata.csv')

In [4]:
# keys to interpret invasive data
key_invasives_status = {'BL':'Black List', 'WL': 'Watch List', 'ODE': 'Prohibited'}
key_invasives_threat = {'1':'low impact','2':'medium impact','3':'high  impact'}
key_invasives_places = {'1':'low frequency','2':'medium frequency','3':'high frequency'}

In [5]:
inv.places

0       [[Jura, 1], [Plateau, 1]]
1       [[Jura, 2], [Plateau, 3]]
2       [[Jura, 3], [Plateau, 3]]
3       [[Jura, 1], [Plateau, 1]]
4       [[Jura, 2], [Plateau, 3]]
5       [[Jura, 1], [Plateau, 1]]
6       [[Jura, 3], [Plateau, 3]]
7       [[Jura, 3], [Plateau, 2]]
8     [[non établi en Suisse, 1]]
9     [[non établi en Suisse, 1]]
10      [[Jura, 1], [Plateau, 2]]
11    [[non établi en Suisse, 1]]
12      [[Jura, 3], [Plateau, 3]]
13      [[Jura, 1], [Plateau, 3]]
14      [[Jura, 3], [Plateau, 3]]
15      [[Jura, 3], [Plateau, 3]]
16    [[non établi en Suisse, 1]]
17      [[Jura, 3], [Plateau, 3]]
18                 [[Plateau, 2]]
19      [[Jura, 1], [Plateau, 2]]
20                 [[Plateau, 1]]
21    [[non établi en Suisse, 1]]
22      [[Jura, 1], [Plateau, 1]]
23      [[Jura, 1], [Plateau, 1]]
24      [[Jura, 1], [Plateau, 2]]
25      [[Jura, 2], [Plateau, 3]]
26      [[Jura, 1], [Plateau, 1]]
27                             []
28      [[Jura, 3], [Plateau, 3]]
29      [[Jura

In [6]:
# note that there are only two threat factors indicated for the plants here.
# the first threat factor 'potential d'expansion' varies little


mylist1 = []
mylist2 = []
for i in inv.threats:
    mylist1.append(int(i[0][1]))
    mylist2.append(int(i[1][1]))
 
inv['expansion'] = mylist1
inv['biodiverse'] = mylist2


In [7]:
inv.iloc[20]

Latein                                                     Ludwigia grandiflora (Michx.) Greuter & Burdet
Black List / Watch List 2014                                                                           BL
Ordonnonce sur la dissémination des organismes (ODE)                                                  ODE
threats                                                 [[Potentiel d'expansion, 3], [écologie, biodiv...
places                                                                                     [[Plateau, 1]]
habitat                                                                                   [1 Eaux libres]
expansion                                                                                               3
biodiverse                                                                                              3
Name: 20, dtype: object

In [28]:
inv.places

0       [[Jura, 1], [Plateau, 1]]
1       [[Jura, 2], [Plateau, 3]]
2       [[Jura, 3], [Plateau, 3]]
3       [[Jura, 1], [Plateau, 1]]
4       [[Jura, 2], [Plateau, 3]]
5       [[Jura, 1], [Plateau, 1]]
6       [[Jura, 3], [Plateau, 3]]
7       [[Jura, 3], [Plateau, 2]]
8     [[non établi en Suisse, 1]]
9     [[non établi en Suisse, 1]]
10      [[Jura, 1], [Plateau, 2]]
11    [[non établi en Suisse, 1]]
12      [[Jura, 3], [Plateau, 3]]
13      [[Jura, 1], [Plateau, 3]]
14      [[Jura, 3], [Plateau, 3]]
15      [[Jura, 3], [Plateau, 3]]
16    [[non établi en Suisse, 1]]
17      [[Jura, 3], [Plateau, 3]]
18                 [[Plateau, 2]]
19      [[Jura, 1], [Plateau, 2]]
20                 [[Plateau, 1]]
21    [[non établi en Suisse, 1]]
22      [[Jura, 1], [Plateau, 1]]
23      [[Jura, 1], [Plateau, 1]]
24      [[Jura, 1], [Plateau, 2]]
25      [[Jura, 2], [Plateau, 3]]
26      [[Jura, 1], [Plateau, 1]]
27                             []
28      [[Jura, 3], [Plateau, 3]]
29      [[Jura

In [33]:
mylist1 = []
mylist2 = []
for i in inv.places:
    if len(i) == 0:
        mylist1.append(0)
        mylist2.append(0)
    if len(i) == 1:
        print(i[0][0])
        if i[0][0] == "Jura":
            mylist1.append(i[0][1])
        if i[0][0] == "Plateau":
            mylist2.append(i[0][1])
        if i[0][0] == "non établi en Suisse":
            mylist1.append(0)
            mylist2.append(0)
        print(mylist1)
    if len(i) == 2:
        print(i[1][0])
        if i[1][0] == "Jura":
            mylist1.append(i[1][1])
        if i[1][0] == "Plateau":
            mylist2.append(i[1][1])
print(mylist1)
inv['plateau'] = mylist1
inv['jura'] = mylist2

Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
non établi en Suisse
[0]
non établi en Suisse
[0, 0]
Plateau
non établi en Suisse
[0, 0, 0]
Plateau
Plateau
Plateau
Plateau
non établi en Suisse
[0, 0, 0, 0]
Plateau
Plateau
[0, 0, 0, 0]
Plateau
Plateau
[0, 0, 0, 0]
non établi en Suisse
[0, 0, 0, 0, 0]
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
non établi en Suisse
[0, 0, 0, 0, 0, 0, 0]
non établi en Suisse
[0, 0, 0, 0, 0, 0, 0, 0]
Plateau
Plateau
non établi en Suisse
[0, 0, 0, 0, 0, 0, 0, 0, 0]
non établi en Suisse
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
Plateau
Plateau
Plateau
Plateau
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Plateau
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


ValueError: Length of values (11) does not match length of index (58)

In [ ]:
inv.expansion.mean()

In [ ]:


inv.biodiverse.mean()

In [ ]:
np.sqrt(inv.expansion.var())